# Digitizer for MSX screen 2
_Gilbert Francois Duivesteijn_

In [1]:
import numpy as np
import cv2 as cv
import os
import glob

import matplotlib.pyplot as plt

from lib.msxcolor import MSXColor
from lib.dither import Dither
from lib.adjustment import Adjustment
import lib.color_transform as ct

In [2]:
def floyd_steinberg_colormap(image, src_palette, dst_palette=None):
    """
    err_diff_matrix = [[          *    7/16   ],
                       [   3/16  5/16  1/16   ]]


    """
    if image.ndim != 3 and image.shape[2] != 3:
        raise RuntimeError(f"Image does not have the right dimensions.")
    out = cv.copyMakeBorder(image, 1, 1, 1, 1, cv.BORDER_REPLICATE)
    if dst_palette is not None:
        dst = np.zeros_like(out, dtype=np.uint8)
    else:
        dst = out
    rows, cols, _ = out.shape
    # import pdb; pdb.set_trace()
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            idx, _ = ct.l2_dist(out[i, j, :], src_palette)
            new_color_idx = idx[0]
            if new_color_idx == 0:
                new_color_idx = 1
            new_color_ij = src_palette[new_color_idx]
            err = out[i][j] - new_color_ij
            out[i][j] = new_color_ij
            dst[i][j] = dst_palette[new_color_idx]
            # out[i:i+2, j-1:j+2] = out[i:i+2, j-1:j+2] + err_diff * err
            out[i    ][j + 1] = out[i    ][j + 1] + (Dither._7o16 * err)
            out[i + 1][j - 1] = out[i + 1][j - 1] + (Dither._3o16 * err)
            out[i + 1][j    ] = out[i + 1][j    ] + (Dither._5o16 * err)
            out[i + 1][j + 1] = out[i + 1][j + 1] + (Dither._1o16 * err)
    out = np.clip(out, 0, 1)
    out = (out*255).astype(np.uint8)
    return (out[1:rows - 1, 1:cols - 1])

In [3]:
image_folder = "../../../resources/test_images"
image_paths = sorted(glob.glob(os.path.join(image_folder, "image*.png")))
image_paths

['../../../resources/test_images/image1.png',
 '../../../resources/test_images/image2.png',
 '../../../resources/test_images/image3.png',
 '../../../resources/test_images/image4.png']

In [4]:
images = [cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB) for image_path in image_paths]

In [5]:
# fig, axs = plt.subplots(2, 2, figsize=(16, 12))
# axs[0][0].imshow(images[0])
# axs[0][1].imshow(images[1])
# axs[1][0].imshow(images[2])
# axs[1][1].imshow(images[3])
# plt.show()

In [6]:
msxcolor = MSXColor()

RGB


NameError: name 'src' is not defined

In [ ]:
images_sc2 = []

params = {
    "style": 1,
    "dither": Dither.JARVIS_JUDICE_NINKE,
    "contrast": 20,
    "hue": 1.0,
    "sat": 1.0,
    "lum": 1.0
}

for i in range(len(images)):
    image_sc2 = msxcolor.screen2(images[i], params=params)
    images_sc2.append(image_sc2)

fig, axs = plt.subplots(2, 4, figsize=(16, 6))
for i in range(len(images)):
    axs[0][i].imshow(images[i])
    axs[1][i].imshow(images_sc2[i])
plt.show()

In [ ]:
params = {
    "style": 2,
    "contrast": 20,
    "hue": 1.0,
    "sat": 1.0,
    "lum": 0.7
}
for i in range(len(images)):
    image_sc2 = msxcolor.screen2(images[i], params=params)
    images_sc2.append(image_sc2)

fig, axs = plt.subplots(2, 4, figsize=(16, 6))
for i in range(len(images)):
    axs[0][i].imshow(images[i])
    axs[1][i].imshow(images_sc2[i])
plt.show()

In [ ]:
msx_color = MSXColor()
image_list = images
for src in image_list:
    # src = cv.cvtColor(src, cv.COLOR_BGR2RGB)
    src = cv.resize(src, (256,192), interpolation=cv.INTER_LINEAR)
    dst = msx_color.fg_map(src.copy(), {"hue": 1.0, "sat": 1.0, "lum": 1.0}, msx_color.palette_msx1_labf, ct.rgbi2labf)
    dst2 = msx_color.fg_map(src.copy(), {"hue": 1.0, "sat": 1.0, "lum": 1.0}, msx_color.palette_msx1_hsvf_xy, ct.rgbi2hsvf_xy)
    dst3 = msx_color.fg_map(src.copy(), {"hue": 1.0, "sat": 1.0, "lum": 1.0}, msx_color.palette_msx1_ycrcbf, ct.rgbi2ycrcbf)
    title_list = ["org", "labf", "hsvf_xy", "rgbf"]
    # dst2 = msx_color.style2(src.copy(), {"hue": 1.0, "sat": 1.0, "lum": 1.0})
    # dst3 = msx_color.style1(src.copy(), {"contrast": 30, "hue": 1.0, "sat": 1.0, "lum": 1.0})
    msx_color._plot3(src, dst, dst2, dst3, title_list=title_list, output_path=None)

